In [1]:
a = [1.0, 2.0, 1.0]

In [3]:
a[0]

1.0

In [5]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

# 첫텐서 만들어보기

In [6]:
import torch
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [8]:
a[1]

tensor(1.)

In [9]:
float(a[1])

1.0

In [11]:
a[2] = 2.0
a

tensor([1., 1., 2.])

In [14]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

points

tensor([4., 1., 5., 3., 2., 1.])

In [15]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [16]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [18]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [28]:
points.shape

torch.Size([3, 2])

In [29]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [30]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [36]:
points[0, 1]

tensor(1.)

In [32]:
points[0]

tensor([4., 1.])

# 텐서 인덱싱

In [37]:
some_list = list(range(6))
some_list[:] # 리스트의 모든 요소

[0, 1, 2, 3, 4, 5]

In [39]:
print(some_list[1:4]) # 1번 인덱스 ~ 4번 인덱스 전
print(some_list[1:]) # 1번 인덱스 부터 끝까지
print(some_list[:4]) # 첫 번째 요소부터 4번 인덱스 전까지
print(some_list[:-1]) # 리스트의 첫번째 요소부터 마지막 요소 바로 앞까지
print(some_list[1:4:2]) # 1번 인덱스부터 4번 인덱스까지, 두 단계씩 건너뛰면서

[1, 2, 3]
[1, 2, 3, 4, 5]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[1, 3]


In [40]:
print(points[1:]) # 첫번째 이후 모든 행에 대해, 암묵적으로 모든 열 포함
print(points[1:, :]) # 첫번째 이후 모든 행에 대해, 명시적으로 모든 열 포함
print(points[1:, 0]) # 첫 번째 이후 모든 행에 대해 첫째 열만
print(points[None]) # 길이가 1인 차원 추가

tensor([[5., 3.],
        [2., 1.]])
tensor([[5., 3.],
        [2., 1.]])
tensor([5., 2.])
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])


# 이름이 있는 텐서

In [41]:
img_t = torch.randn(3, 5, 5) # 각각이 [채널 크기, 행 크기, 열 크기]가 됨
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [42]:
batch_t = torch.randn(2, 3, 5, 5) # [채널 크기, 행 크기, 열 크기]

In [43]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [44]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [56]:
img_gray_weighted_fancy = torch.einsum('...chw, c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw, c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [57]:
# 이름 지정이 가능한 텐서
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names = ['channels'])
weights_named

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1463.)
  


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [58]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named:', img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [59]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [60]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [61]:
# 이름이 다른 차원을 결합하려면 오류가 발생
# gray_named = (img_named[..., :3] * weights_named).sum("channels")

RuntimeError: ignored

In [62]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

# dtype으로 숫자 정하기
- torch.float32 혹은 torch.float : 32비트 단정밀도 부동소수점
- torch.float64 혹은 torch.double : 64비트 배정밀도 부동소수점
- torch.float16 혹은 torch.half : 16비트 반정밀도 부동소수점
- torch.int8 : 부호 있는 8비트 정수
- torch.uint8 : 부호 없는 8비트 정수
- torch.int16 혹은 torch.short : 부호 있는 16비트 정수
- torch.int32 혹은 torch.int : 부호 있는 32비트 정수
- torch.int64 혹은 torch.long : 부호 있는 64비트 정수
- torch.bool : 불리언

* 텐서의 기본 데이터 타입은 32비트 부동소수점이다.

In [63]:
# 텐서의 dtype 속성 관리
double_points = torch.ones(10, 2, dtype = torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype = torch.short)

In [64]:
short_points.dtype

torch.int16

In [65]:
double_points = torch.zeros(10,2).double()
short_points = torch.ones(10, 2).short()

In [66]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype = torch.short)

In [67]:
points_64 = torch.rand(5, dtype = torch.double)
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [68]:
# 텐서 API
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [73]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))